In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

from os import path
from glob import glob
import itertools
import gc
import pickle
from itertools import chain, combinations

import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from IPython.display import display
from IPython.utils.capture import capture_output
from tqdm.auto import tqdm
with capture_output():
    tqdm.pandas()

from differentiation import spectral_differentiation as specD
from differentiation import spectral_states as specS

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False

In [2]:
data_directory = '/allen/programs/braintv/workgroups/tiny-blue-dot/differentiation/refactor/data'

session_ids = [
    path.basename(x)
    .strip('.pkl')
    .strip('fr_') for x in glob(
        path.join(data_directory, 'fr_*')
    )
]

In [3]:
region_sets = {
    'VisCtx' : ['VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'],
    'HVAs' : ['VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'],
    'THx_VISp' : ['LGd', 'LP', 'TH', 'VISp'],
    'AllVis' : ['LGd', 'LP', 'TH', 'VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'],
    'THx' : ['LGd', 'LP', 'TH'],
    'hipp' : ['CA', 'CA1', 'CA2', 'CA3', 'DG', 'DG-mo', 'DG-po', 'DG-sg'],
}

relevant_regions = [
    'LGd', 'LP', 'TH', 'VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'
]

stimulus_categories = {
    'drifting_gratings' : 'complex',
    'drifting_gratings_contrast' : 'simple',
    'flashes' : 'simple',
    'gabors' : 'simple',
    'natural_movie_one_shuffled' : 'shuffled',
    'natural_movies' : 'natural',
    'natural_movie_one' : 'natural',
    'natural_movie_three' : 'natural',
    'spontaneous' : 'spontaneous',
    'static_gratings' : 'complex'
}

intuitively_ordered_stimuli = {
    'stimulus_name' : [
        'spontaneous', 'natural_movie_one_shuffled', 'flashes', 'gabors', 'drifting_gratings_contrast',
        'drifting_gratings', 'static_gratings', 'natural_movie_one', 'natural_movie_three', 'natural_movies'
    ],
    'stimulus_category' : ['shuffled', 'spontaneous', 'simple', 'complex', 'natural']
}

layer_order = ['L2/3', 'L4', 'L5', 'L6']

hierarchy = {
    'Input' : -100,
    'stimulus' : -100,
    'Stim' : -100,
    'TH' : -10,
    'LG' : -9,
    'LGv' : -8,
    'LGd' : -7,
    'LP' : -6,
    'THx' : -5,
    'THx_VISp' : -4,
    'VISp' : 0,
    'VISpl' : 2,
    'VISl' : 4,
    'VISli' : 6,
    'VISrl' : 8,
    'VISal' : 10,
    'VISpm' : 12,
    'VISam' : 14,
    'VISpor' : 16,
    'VISa' : 18,
    'SC' : 24,
    'VISmma' : 20,
    'VISmmp' : 20,
    'VIS' : 22,
    'HVAs' : 21,
    'VisCtx' : 21.5,
    'AllVis' : 22,
    'PF' : 25,
    'MB' : 30,
    'hipp' : 38,
    'CAx' : 39,
    'CA' : 40,
    'CA1' : 41,
    'CA2' : 42,
    'CA3' : 43,
    'DG' : 50,
}

In [4]:
# set up parameters for computing spectral differentiatioon
state_length_scaling_exponent = 2
sampling_rate = 200
state_lengths = [0.005, 0.01, 0.015, 0.03, 0.06, 0.1, 0.15, 0.3, 0.5, 1]
win_lengths = [3, 9, 30]
param_sets = [
    {
        'STATE_LENGTH' : s,
        'WINDOW' : w,
        'RESOLUTION' : w, # not used, defaults to w in this notebook
    } for w, s in itertools.product(win_lengths, state_lengths) if w/s<=600
]
print(f'Computing for {len(param_sets)} parameter sets per session per ensemble.')

Computing for 24 parameter sets per session per ensemble.


In [5]:
def load_fr(session):
    return pd.read_pickle(
        path.join(data_directory, f'fr_{session}.pkl')
    )

def load_units(session):
    return pd.read_pickle(
        path.join(data_directory, f'units_{session}.pkl')
    )

def load_stimulus_table(session):
    return pd.read_pickle(
        path.join(data_directory, f'stimulus_{session}.pkl')
    )

In [6]:
def get_specD(fr, units, times, window_length, state_length, sampling_rate=200):
    fr_win = fr[:int(
        fr.shape[0]/(window_length*sampling_rate)
    )*window_length*sampling_rate]
    fr_win = np.reshape(
        fr_win.T,
        (fr_win.shape[1], -1, window_length*sampling_rate)
    ).transpose(1, 0, 2)[:, units, :]
    # compute spectral differentiation
    try:
        df = specD(
            fr_win, sample_rate=sampling_rate, window_length=state_length
        )
    except Exception as e:
        print(e)
        df = np.zeros((fr.shape[0], 2))
    # get median differentiation and normalize with number of units
    df = np.median(df, axis=1) / np.sqrt(len(units))
    
    _times = np.linspace(
        times[0],
        int(times[-1]/window_length)*int(window_length),
        df.shape[0], False, dtype=int
    ) + window_length/2
#     _times = np.linspace(times[0], times[-1], df.shape[0], False)
#     _times = _times + np.diff(_times).mean()/2
    return pd.Series(
        df, index=pd.MultiIndex.from_frame(
            pd.MultiIndex.from_frame(stim_table)
            .drop_duplicates()
            .to_frame(index=False)
            .set_index('time')
            .reindex(
                _times, method='ffill'
            ).rename_axis('time').reset_index().bfill()
        ),
        name=(window_length, state_length)
    ).rename_axis(stim_table.columns)

def get_specD_individual_units(
    fr, window_length, state_length, times, stim_table,
    sampling_rate=200, sort=True, col_offset=0
):
    fr_win = fr[:int(
        fr.shape[0]/(window_length*sampling_rate)
    )*window_length*sampling_rate]
    fr_win = np.reshape(
        fr_win.T,
        (fr_win.shape[1], -1, window_length*sampling_rate)
    )
    df = specD(
        fr_win[:, :, np.newaxis, :],
        sample_rate=sampling_rate,
        window_length=state_length
    )
    df = np.median(df, axis=-1)
    if sort:
        df = df[np.argsort(fr.mean(0))]
    
#     display(df.shape)
    _times = np.linspace(
        times[0],
        int(times[-1]/window_length)*int(window_length),
        df.shape[-1], False, dtype=int
    ) + window_length/2
#     _times = np.linspace(times[0], times[-1], df.shape[-1], False)
#     _times = _times + np.diff(_times).mean()/2
    df = pd.DataFrame(
        df.T, index=pd.MultiIndex.from_frame(
            pd.MultiIndex.from_frame(stim_table)
            .drop_duplicates()
            .to_frame(index=False)
            .set_index('time')
            .reindex(
                _times, method='ffill'
            ).rename_axis('time').reset_index().bfill()
        )
    ).rename_axis(stim_table.columns)
    df.columns = df.columns + col_offset
    df.columns = pd.MultiIndex.from_product(
        [df.columns, [window_length], [state_length]],
        names=['unit', 'window_length', 'state_length']
    )
    return df

In [7]:
def gen_staggered_ts(_fr, _times, shift_ms=5, shift_var=0):
    return np.interp(_times-shift_ms+np.random.rand(*_times.shape)*shift_var, _times, _fr)

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [8]:
def get_autocorr(arr):
    arr = arr - arr.mean()
    x = np.correlate(arr, arr, mode='full')
    x = x[x.size//2:]/x.max()
    return x

def exp(x, t, a):
    return a*np.exp(-x/t)

def get_autocorr_time(arr, sampling_rate=200):
    ac = get_autocorr(arr)
    _x = np.linspace(0, 1, sampling_rate, False)
    T, _a = sp.optimize.curve_fit(exp, _x, ac[:sampling_rate])[0]
    return T, _a

# Single neuron differentiation wrt window and state length

In [9]:
session = session_ids[6]
units = load_units(session)
display(units[units.snr>2.5].groupby('region').size().sort_values(ascending=False))
stim_table = load_stimulus_table(session)
display(stim_table.stimulus_name.unique())

region
VIS       134
CA1       114
VISp       84
VISam      67
VISrl      66
SUB        48
APN        47
LP         32
VISmmp     20
DG         17
LGv        11
CA3         8
SGN         6
MB          6
NOT         3
ProS        1
PPT         1
dtype: int64

array(['spontaneous', 'gabors', 'flashes', 'drifting_gratings',
       'natural_movie_three', 'natural_movie_one', 'static_gratings',
       'natural_scenes', 'drifting_gratings_contrast'], dtype=object)

In [10]:
fr = load_fr(session)

units = load_units(session)
units['idx'] = range(len(units))

times = fr.index

stim_table = load_stimulus_table(session)

area = 'VISp'

In [11]:
# fr = fr.loc[:, units.idx[(units.snr>2.5)&(units.region==area)].values]
# fr_mean = fr.mean()
# fr = fr / fr.mean() # normalize (for consistency, but should not matter within an area)
# display(fr.shape)

# units = {
#     'set1' : np.argsort(fr_mean.values)[-7:],
#     'all' : range(len(fr_mean))
# }
# iunits = list(np.argsort(fr_mean.values)[-7:])

# f, axes = plt.subplots(
#     round((len(units)+len(iunits)+1.49)/3), 3,
#     figsize=(12, 2.3*round((len(units)+len(iunits)+1.49)/3)),
#     tight_layout=True, sharex=True
# )

# idf = get_specD_individual_units(fr.values, 3, 0.1, times, stim_table, sort=False)

# for i, (ax, unit) in enumerate(zip(axes.flatten(), iunits+list(units.items()))):
#     if hasattr(unit, '__len__'):
#         df = get_specD(fr.values, unit[1], 3, 0.1)
#         ax.set_title(unit[0])
#     else:
#         df = idf[unit]
#         ax.set_title(f'unit {unit}')
    
#     df.groupby('stimulus_name').apply(
#         lambda d: d.reset_index().set_index('time').iloc[1:, -1]
#         .plot(ax=ax, lw=0, marker='.', ms=2, label=d.name)
#     )
#     ax.legend(fontsize=5, loc=0)

---

In [12]:
aois = ['VISam', 'VISp']

fn = path.join(
    path.dirname(data_directory),
    f'timescales/{session}_single_unit_dfn.pkl'
)

fr = load_fr(session)

units = load_units(session)
units['idx'] = range(len(units))

times = fr.index

stim_table = load_stimulus_table(session)

df = {}
for area in [
    c for c in units.region.unique() if c in aois
]:
    _fr = fr.values[
        :, units.idx[(units.snr>2.5)&(units.region==area)].values
    ]
    _fr = _fr / _fr.mean()
    for state in tqdm(state_lengths[::-1]):
        for win in [3]:#win_lengths:
            if win / state > 600:
                continue # skip very extreme conditions
            df[
                (win, state, area, '-', True, 'all')
            ] = get_specD(
                _fr, range(_fr.shape[1]), times, win, state
            ).drop_duplicates()
#             display(df[
#                 (win, state, area, '-', True, 'all')
#             ])
differentiation = pd.concat(
    df, axis=1,
    names=[
        'window_length', 'state_length',
        'area', 'layer', 'FS_RS', 'unit'
    ]
).stack(
    ['area', 'layer', 'FS_RS', 'unit']
).append(pd.read_pickle(fn)).sort_index()

differentiation = differentiation / differentiation.columns.get_level_values('state_length')**state_length_scaling_exponent
differentiation = differentiation.unstack(['unit', 'area', 'layer', 'FS_RS']).dropna(how='all')
differentiation

window_length                                3                                \
state_length                             0.005                                 
unit                                         0                                 
area                                        LP                                 
layer                                        -          L1        L2/3         
FS_RS                                    False True  False True  False True    
time    stimulus_name              block                                       
1.5     spontaneous                -1.0    NaN   NaN   NaN   NaN   NaN   NaN   
4.5     spontaneous                -1.0    NaN   NaN   NaN   NaN   NaN   NaN   
7.5     spontaneous                -1.0    NaN   NaN   NaN   NaN   NaN   NaN   
10.5    spontaneous                -1.0    NaN   NaN   NaN   NaN   NaN   NaN   
13.5    spontaneous                -1.0    NaN   NaN   NaN   NaN   NaN   NaN   
...                                        ...   ...   ...   ...   ...   ...   
10192.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN   NaN   NaN   
10195.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN   NaN   NaN   
10198.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN   NaN   NaN   
10201.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN   NaN   NaN   
10204.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN   NaN   NaN   

window_length                                                     ...        \
state_length                                                      ... 1.000   
unit                                                              ...   all   
area                                                              ... VISrl   
layer                                       L4          L5        ...    L1   
FS_RS                                    False True  False True   ... False   
time    stimulus_name              block                          ...         
1.5     spontaneous                -1.0    NaN   NaN   NaN   NaN  ...   NaN   
4.5     spontaneous                -1.0    NaN   NaN   NaN   NaN  ...   NaN   
7.5     spontaneous                -1.0    NaN   NaN   NaN   NaN  ...   NaN   
10.5    spontaneous                -1.0    NaN   NaN   NaN   NaN  ...   NaN   
13.5    spontaneous                -1.0    NaN   NaN   NaN   NaN  ...   NaN   
...                                        ...   ...   ...   ...  ...   ...   
10192.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN  ...   NaN   
10195.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN  ...   NaN   
10198.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN  ...   NaN   
10201.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN  ...   NaN   
10204.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN  ...   NaN   

window_length                                                                 \
state_length                                                                   
unit                                                                           
area                                                                           
layer                                           L2/3          L4          L5   
FS_RS                                    True  False True  False True  False   
time    stimulus_name              block                                       
1.5     spontaneous                -1.0    NaN   NaN   NaN   NaN   NaN   NaN   
4.5     spontaneous                -1.0    NaN   NaN   NaN   NaN   NaN   NaN   
7.5     spontaneous                -1.0    NaN   NaN   NaN   NaN   NaN   NaN   
10.5    spontaneous                -1.0    NaN   NaN   NaN   NaN   NaN   NaN   
13.5    spontaneous                -1.0    NaN   NaN   NaN   NaN   NaN   NaN   
...                                        ...   ...   ...   ...   ...   ...   
10192.5 drifting_gratings_contrast  15.0   NaN   NaN   NaN   NaN   NaN   NaN   
10195.5 drifting

In [13]:
# we will plot those units that do have an optimal timescale within 0.01 s to 1 s
mdf = differentiation.mean().unstack(['window_length', 'state_length'])
idxmax = mdf[3].idxmax(1).rename('idxmax')
# remove instances where the first or last state length is the optimal
idxmax.replace([0.01], -1, inplace=True)
idxmax.replace([1], -2, inplace=True)
# remove instances where the max and min are same (degenerate case)
idxmax[mdf[3].max(1)==mdf[3].min(1)] = -3
idxmax[idxmax<0] = np.nan
idxmax

unit  area   layer  FS_RS
0     LP     -      False   NaN
                    True    NaN
             L1     False   NaN
                    True    NaN
             L2/3   False   NaN
                             ..
all   VISrl  L4     True    NaN
             L5     False   NaN
                    True    NaN
             L6     False   NaN
                    True    NaN
Name: idxmax, Length: 4080, dtype: float64

In [41]:
stim_mean_diff = {}
for area in aois:
    stim_mean_diff[area] = differentiation.xs(
        area, level='area', axis=1
    ).groupby(['stimulus_name']).mean()
    stim_mean_diff[area][stim_mean_diff[area]<1] = 0

stim_std_diff = {}
for area in aois:
    stim_std_diff[area] = (differentiation.xs(
        area, level='area', axis=1
    ).groupby(['stimulus_name']).std().T / differentiation.xs(
        area, level='area', axis=1
    ).groupby(['stimulus_name']).size().map(np.sqrt)).T
    stim_std_diff[area][stim_mean_diff[area]<1] = 0

# plot_units = {
#     k : list(
#         v.mean().groupby('unit').mean().dropna()
#         .sort_values().index[-8:]
#     ) for k, v in stim_mean_diff.items()
# }

# plot_units = {
#     k : fr.values[
#         :, units.index[(units.snr>2.5)&(units.region==k)].values
#     ].mean(0).argsort()[-8:] for k in stim_mean_diff.keys()
# }

# plot_units = {
#     k : list(
#         v.mean().loc[(3, 0.1)].dropna()
#         .sort_values().index[-8:]
#         .get_level_values('unit')
#     ) for k, v in stim_mean_diff.items()
# }

# plot units that have an optima, and high differentiation (for visual clarity)
plot_units = {
    k : list(
        stim_mean_diff[k].mean()
        .loc[(3, 0.1)].dropna().loc[
            idxmax.dropna().xs(k, level='area')
            .index.get_level_values('unit')
        ].sort_values().index[-8:]
        .get_level_values('unit')
    ) for k, v in stim_mean_diff.items()
}

_stim_mean_diff_plt = {
    k : v.reorder_levels(
        [2, 0, 1, 3, 4], axis=1
    )[plot_units[k]].reorder_levels(
        [1, 2, 0, 3, 4], axis=1
    ).dropna(1) for k, v in stim_mean_diff.items()
}

_stim_std_diff_plt = {
    k : v.reorder_levels(
        [2, 0, 1, 3, 4], axis=1
    )[plot_units[k]].reorder_levels(
        [1, 2, 0, 3, 4], axis=1
    ).dropna(1) for k, v in stim_std_diff.items()
}

stim_mean_diff[area][3][0.3].dropna(how='all', axis=1)

unit,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,all
layer,L1,L1,L1,L2/3,L2/3,L2/3,L2/3,L2/3,L2/3,L2/3,...,L5,L5,L5,L5,L6,L6,L6,L6,L6,-
FS_RS,True,True,True,True,True,False,True,True,True,False,...,True,True,True,True,True,True,True,False,True,True
stimulus_name,,,,,,,,,,,,,,,,,,,,,
drifting_gratings,95115.673430,147542.902467,409.874067,200703.716216,6946.203153,6.040094e+05,142726.407972,555977.688035,4591.233845,4.286162e+05,...,6.367242,0.000000,21462.241372,257.488660,8363.369905,2159.786394,26.627221,238.521222,1137.118108,1.002662e+06
drifting_gratings_contrast,547953.585023,89230.017139,261.295906,24017.571523,686.271948,9.112771e+05,24054.380113,23350.487092,12518.755639,1.470997e+06,...,3922.992541,2052.509550,14729.799623,489.649230,1146.051144,6118.821520,51.714949,673.993042,10853.323823,8.756444e+05
flashes,245019.991749,138397.903654,0.000000,0.000000,2205.299999,5.741927e+05,3242.048426,7737.272489,68357.906902,3.539592e+05,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2343.571410,5.417808e+05
gabors,122424.683188,144527.286285,0.000000,0.000000,344.202340,4.399155e+05,7.660415,6384.622794,0.000000,1.618481e+05,...,0.000000,0.000000,0.000000,0.000000,533.672777,0.000000,0.000000,0.000000,4636.547527,4.953109e+05
natural_movie_one,344682.331550,251470.716596,144755.279230,22140.382154,2983.742297,1.130220e+06,14580.334606,25147.496782,51.693828,1.455402e+06,...,3769.459443,49582.691978,10726.544689,3717.454044,96351.174999,15715.956039,0.000000,49997.061539,22210.276754,1.108203e+06
natural_movie_three,241071.082598,267293.869210,29726.350662,14594.050206,5881.092496,1.350395e+06,3366.754851,27652.023598,1948.304603,7.791355e+05,...,2361.447501,2563.221678,5842.661194,3525.386978,50407.608491,126347.725965,4760.336761,9293.259146,34815.028017,9.390739e+05
natural_scenes,404061.245954,144798.425778,0.000000,140565.194023,51593.084821,5.981418e+05,502746.393496,34317.428403,37133.814726,7.543969e+05,...,3244.225794,929.895554,26564.567655,10127.072537,85638.116089,8919.105641,4584.524085,8334.902053,98816.478997,1.254191e+06


In [42]:
f, axes = plt.subplots(
    1, 4, figsize=(7.55, 1.7), constrained_layout=True,
    sharex=True, sharey=True
)
i = 0
for area in aois:
    wins = stim_mean_diff[area].columns.levels[0]
    for stim in ['spontaneous', 'static_gratings']:
        ax = axes[i]
        for j, win in enumerate(reversed([wins])):
            _stim_mean_diff_plt[area].loc[stim, win].groupby(#.drop('all', axis=1, level='unit')
                'unit', sort=False
            ).apply(
#                 lambda d: display(_stim_std_diff_plt[area].loc[stim, win].xs(d.name, level=2), d.droplevel(['unit', 'layer', 'FS_RS', 'window_length']))
                lambda d: d.droplevel(['unit', 'layer', 'FS_RS', 'window_length']).dropna().plot(
                    marker='o', ax=ax, label=f'{"_"*(len(wins)-1-j)}{d.name}',
                    color=f'C{sorted([str(x) for x in plot_units[area]]).index(str(d.name))}',
                    alpha=0.6-0.4*j/len(wins) if d.name!='all' else 0.9, markersize=2,
                    yerr=_stim_std_diff_plt[area].loc[stim, win].xs(d.name, level=2).droplevel([0, 2, 3])
                )
            )
        ax.set_xscale('log')
        ax.set_yscale('log')
#         ax.set_xlim(0.007, 1.2)
#         ax.set_ylim(1e5, 1e7)
        ax.tick_params(axis='both', which='major', labelsize=7)
        ax.set_xlabel('state length (s)', fontsize=8)
        ax.set_ylabel('differentiation', fontsize=8)
        ax.set_title(f'{area} {stim}', fontsize=8)
        i += 1
# f.savefig('fig_timescales_neurons.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
f.savefig('fig_timescales_neurons.pdf')

# Get t_opt for all visual units for all stimuli for all experiments

In [9]:
def single_unit_fr_by_session(session, SNR=2.5):
    fn = path.join(
        path.dirname(data_directory),
        f'timescales/{session}_single_unit_fr.pkl'
    )
    try:
#         zzz
        return pd.read_pickle(fn)
    except:
        fr = load_fr(session)

        units = load_units(session)
        units['idx'] = range(len(units))

        times = fr.index
        _times = np.linspace(
            times[0], times[-1], fr.shape[0], False
        )
        _times = _times + np.diff(_times).mean()/2

        stim_table = load_stimulus_table(session)
        
        frs = {}
        for area in tqdm(
            units[
                (units.snr>SNR)&(units.region.isin(relevant_regions))
            ].region.unique(), desc='area'
        ):
            _fr = fr.values[
                :, units.idx[(units.snr>SNR)&(units.region==area)].values
            ]
            _frs = pd.DataFrame(
                _fr, index=pd.MultiIndex.from_frame(
                    pd.MultiIndex.from_frame(stim_table)
                    .drop_duplicates()
                    .to_frame(index=False)
                    .set_index('time')
                    .reindex(
                        _times, method='ffill'
                    ).rename_axis('time').reset_index().bfill()
                )
            ).rename_axis(stim_table.columns)
            _frs.columns = pd.MultiIndex.from_frame(
                _frs.columns.to_frame(index=False).join(
                    units[
                        (units.snr>SNR)&(units.region==area)
                    ].layer.reset_index(drop=True)
                ).join(
                    units[
                        (units.snr>SNR)&(units.region==area)
                    ].RS.reset_index(drop=True)
                )
            )
            frs[area] = _frs
        frs = pd.concat(frs, axis=1, names=['area', 'unit', 'layer', 'FS_RS'])
        frs = frs.rolling(3*200, center=True).mean().iloc[3*100:].iloc[::3*200]
        frs = frs.stack(['area', 'layer', "FS_RS", 'unit'], dropna=False)
        frs.to_pickle(fn)
        return frs

In [10]:
def single_unit_differentiation_by_session(session, SNR=2.5):
    fn = path.join(path.dirname(data_directory), f'timescales/{session}_single_unit_dfn.pkl')
    try:
#         zzz
        return pd.read_pickle(fn)
    except:
        fr = load_fr(session)

        units = load_units(session)
        units['idx'] = range(len(units))

        times = fr.index

        stim_table = load_stimulus_table(session)
        
        idf = {}
        for area in tqdm(units[(units.snr>SNR)&(units.region.isin(relevant_regions))].region.unique(), desc='area'):
#             print(area)
            _fr = fr.values[:, units.idx[(units.snr>SNR)&(units.region==area)].values]
            _fr = _fr / _fr.mean()
            for state in state_lengths[1:-1]:
                for win in [3]:
                    _idf = []
                    for w in range((_fr.shape[1]+9)//10):
                        try:
                            _idf.append(
                                get_specD_individual_units(
                                    _fr[:, w*10:(w+1)*10], win, state,
                                    times, stim_table, sort=False, col_offset=w*10
                                ).reorder_levels([1, 2, 0], axis=1)
                            )
                        except Exception as e:
                            print(f'failed for {session} {win} {state}: {e}')
                    idf[(area, win, state)] = pd.concat(_idf, axis=1)
                    idf[(area, win, state)].columns = pd.MultiIndex.from_frame(
                        idf[(area, win, state)].columns.to_frame(index=False).join(
                            units[(units.snr>SNR)&(units.region==area)].layer.reset_index(drop=True)
                        ).join(
                            units[(units.snr>SNR)&(units.region==area)].RS.reset_index(drop=True)
                        )
                    ).droplevel([0, 1])
        df = pd.concat(idf, axis=1, names=['area', 'window_length', 'state_length', 'unit', 'layer', 'FS_RS'])
#         return df
        df.columns = pd.MultiIndex.from_frame(df.columns.to_frame().fillna('-'))
        df = df.stack(['area', 'layer', 'FS_RS', 'unit'])
        df.to_pickle(fn)
        return df

In [11]:
fn = path.join(path.dirname(data_directory), 'timescales', 'all_single_units_frs.pkl')
if path.exists(fn):
    frs = pd.read_pickle(fn)
else:
    frs = {}
    for session in tqdm(session_ids[::-1]):
        frs[session] = single_unit_fr_by_session(session)
    frs = pd.concat(frs)
    frs.to_pickle(fn)
frs = frs.rename_axis(['session']+list(frs.index.names)[1:])
mfrs = frs.groupby(['session', 'stimulus_name', 'unit', 'area', 'layer', 'FS_RS']).mean().rename('mean firing rate')
mfrs

session    stimulus_name      unit  area   layer  FS_RS
763673393  drifting_gratings  0     VISam  L2/3   False     0.074450
                                    VISl   L1     False     0.371894
                                    VISp   L1     False     0.850910
                                    VISrl  L1     False    10.188595
                              1     VISam  L2/3   True      2.275603
                                                             ...    
799864342  static_gratings    59    VISp   L6     True      0.094633
                              60    VISp   L6     True      0.062180
                              61    VISp   L6     True      2.574340
                              62    VISp   L6     True      0.433113
                              63    VISp   L6     True      7.823037
Name: mean firing rate, Length: 134972, dtype: float64

In [12]:
fn = path.join(path.dirname(data_directory), 'timescales', 'all_single_units_dfn.pkl')
if path.exists(fn):
    dfn = pd.read_pickle(fn)
else:
    print('Generating concatenated dataframe.')
    df = {}
    for session in tqdm(session_ids):
        df[session] = single_unit_differentiation_by_session(session)
    dfn = pd.concat(df)
    dfn.to_pickle(fn)
dfn = dfn / dfn.columns.get_level_values('state_length')**state_length_scaling_exponent
dfn = dfn.rename_axis(['session']+list(dfn.index.names)[1:])
mdfn = dfn.groupby(['session', 'stimulus_name', 'unit', 'area', 'layer', 'FS_RS']).mean()
mdfn

window_length                                                   3  \
state_length                                                0.010   
session   stimulus_name     unit area  layer FS_RS                  
763673393 drifting_gratings 0    LGd   -     True   114077.599766   
                                 LP    -     False  274738.917698   
                                 TH    -     False       0.000000   
                                 VISam L2/3  False       0.000000   
                                 VISl  L1    False       0.000000   
...                                                           ...   
799864342 static_gratings   63   LP    -     False  141141.717238   
                                 VISp  L6    True    73624.767629   
                            64   LP    -     False  566232.652251   
                            65   LP    -     True        0.000000   
                            66   LP    -     False  646861.729628   

window_length                                                      \
state_length                                                0.015   
session   stimulus_name     unit area  layer FS_RS                  
763673393 drifting_gratings 0    LGd   -     True   130906.820665   
                                 LP    -     False  265653.089022   
                                 TH    -     False       0.000000   
                                 VISam L2/3  False       0.000000   
                                 VISl  L1    False       0.000000   
...                                                           ...   
799864342 static_gratings   63   LP    -     False  138292.569303   
                                 VISp  L6    True    80350.905491   
                            64   LP    -     False  555143.383383   
                            65   LP    -     True        0.000000   
                            66   LP    -     False  655541.946065   

window_length                                                      \
state_length                                                0.030   
session   stimulus_name     unit area  layer FS_RS                  
763673393 drifting_gratings 0    LGd   -     True   194283.087032   
                                 LP    -     False  252268.262517   
                                 TH    -     False       0.000000   
                                 VISam L2/3  False       0.000000   
                                 VISl  L1    False       0.000000   
...                                                           ...   
799864342 static_gratings   63   LP    -     False  134219.365742   
                                 VISp  L6    True   142270.892354   
                            64   LP    -     False  538145.052234   
                            65   LP    -     True        0.000000   
                            66   LP    -     False  609009.995514   

window_length                                                      \
state_length                                                0.060   
session   stimulus_name     unit area  layer FS_RS                  
763673393 drifting_gratings 0    LGd   -     True   195768.488504   
                                 LP    -     False  228387.744612   
                                 TH    -     False     272.935222   
                                 VISam L2/3  False       0.000000   
                                 VISl  L1    False       0.000000   
...                                                           ...   
799864342 static_gratings   63   LP    -     False  128146.620826   
                                 VISp  L6    True   241976.374887   
                            64   LP    -     False  458289.588244   
                            65   LP    -     True        0.000000   
                            66   LP    -     False  545526.150010   

window_length                                                      \
state_length                                                0.1

In [13]:
mdfn[3].reorder_levels([0, 3, 4, 5, 2, 1]).sort_index().loc[('754312389')].loc['VISp'].loc['L5'].loc[True]

state_length                      0.010          0.015          0.030  \
unit stimulus_name                                                      
29   drifting_gratings       659.511280     972.813364    4956.445763   
     flashes                  24.444584     181.815144    2859.759757   
     gabors                    0.000000       0.000000       0.625442   
     natural_movie_one     19591.830192   26242.090491   76675.821094   
     natural_movie_three  180156.358243  223636.335060  324040.647938   
...                                 ...            ...            ...   
84   natural_movie_one         0.000000       0.000000      25.834829   
     natural_movie_three       0.000000       0.000000      12.917415   
     natural_scenes            0.000000       0.000000       0.000000   
     spontaneous               0.000000       0.000000       0.000000   
     static_gratings           0.000000       0.000000       0.000000   

state_length                      0.060          0.100          0.150  \
unit stimulus_name                                                      
29   drifting_gratings     23910.900068   65315.921568   86620.253643   
     flashes               40667.188593   82872.165135  114546.345896   
     gabors                 3103.465258   10582.359278   20819.056968   
     natural_movie_one    245797.074607  288594.701511  291987.599157   
     natural_movie_three  419860.928298  459006.741427  450746.189797   
...                                 ...            ...            ...   
84   natural_movie_one      2705.263519    6917.492500    9253.533645   
     natural_movie_three     438.848363    3566.403347    6281.192888   
     natural_scenes            0.000000       0.000000     307.146879   
     spontaneous               0.000000     187.253878     569.124430   
     static_gratings           0.093157     134.807246     501.552192   

state_length                      0.300          0.500          1.000  
unit stimulus_name                                                     
29   drifting_gratings    110766.053522  110162.967692  136481.998928  
     flashes              131677.973383  119834.692305  149231.806465  
     gabors                51376.397278   55671.559812   66883.240456  
     natural_movie_one    295071.827226  303513.566293  327446.376607  
     natural_movie_three  416821.793021  428400.470920  499946.775753  
...                                 ...            ...            ...  
84   natural_movie_one     22911.845980   26446.836717   62378.599462  
     natural_movie_three   18555.471638   24392.982932   61182.652800  
     natural_scenes         5227.824043   12300.862688   23331.712126  
     spontaneous            1726.385658    4437.582282    9820.370414  
     static_gratings        1078.172108    4555.015688   10798.237296  

[168 rows x 9 columns]

## Same neuron, different stimuli (comparison)

In [15]:
f, ax = plt.subplots(figsize=(6, 3.6), tight_layout=True)
mdfn[3].reorder_levels([0, 2, 3, 4, 5, 1]).sort_index().loc[('754312389', 29, 'VISp', 'L5', True)].T.plot(ax=ax)
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(fontsize=6);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# -1: max occurred at 0.01s
# -2: max occurred at 1s
# -3: max and min are the same (degenerate point)

dfnmax = mdfn[3].max(1).rename('dfnmax')

idxmax = mdfn[3].idxmax(1).rename('idxmax')
# remove instances where the first or last state length is the optimal
idxmax.replace([0.01], -1, inplace=True)
idxmax.replace([1], -2, inplace=True)
# remove instances where the max and min are same (degenerate case)
idxmax[mdfn[3].max(1)==mdfn[3].min(1)] = -3

idxmax_grouped = idxmax.copy()
idxmax_grouped[idxmax_grouped>0] = 1

idxmax

session    stimulus_name      unit  area   layer  FS_RS
763673393  drifting_gratings  0     LGd    -      True     0.100
                                    LP     -      False   -1.000
                                    TH     -      False   -2.000
                                    VISam  L2/3   False   -2.000
                                    VISl   L1     False   -2.000
                                                           ...  
799864342  static_gratings    63    LP     -      False   -2.000
                                    VISp   L6     True     0.150
                              64    LP     -      False   -1.000
                              65    LP     -      True    -2.000
                              66    LP     -      False    0.015
Name: idxmax, Length: 171371, dtype: float64

In [17]:
_idx = idxmax.index.get_level_values('stimulus_name').isin([
    'drifting_gratings_75_repeats',
    'drifting_gratings_contrast',
    'flashes', 'gabors', 'spontaneous',
    'natural_movie_one', 'natural_movie_one_shuffled'
]) & idxmax.index.get_level_values('layer').isin([
    'L1', 'L2/3', 'L4', 'L5', 'L6'
])

In [18]:
# most neurons that do not have an optimal timescale
# have peak differentiation at the longest explored timescale
# of 1 s

(
    idxmax_grouped[_idx].to_frame().groupby(['idxmax', 'FS_RS', 'layer']).size() \
    / idxmax_grouped[_idx].groupby('FS_RS').size() * 100
).loc[[-3, -2, -1, 1]].unstack(0).rename(
    columns={-1:'0.01 s', -2:'1 s', -3:'degenerate', 1:'optimal'}
)

idxmax       degenerate        1 s    0.01 s    optimal
FS_RS layer                                            
False L1       4.348120  10.494996  2.346497   9.609143
      L2/3     4.530701  11.982689  4.246687  11.867731
      L4       0.743846   3.577225  1.879903   4.943197
      L5       2.021910   7.357317  2.623749   9.304842
      L6       0.845280   3.435218  0.710035   3.130917
True  L1       0.730288   3.671215  0.108766   1.849027
      L2/3     2.534113  14.256858  0.274035   5.192530
      L4       1.470464   7.205413  0.824929   9.181569
      L5       2.809560  19.487527  0.430827   8.270475
      L6       3.213549  17.083369  0.062152   1.343334

## Optimal time vs max differentiation

In [19]:
idxmax[idxmax<0] = np.nan
idxmax

session    stimulus_name      unit  area   layer  FS_RS
763673393  drifting_gratings  0     LGd    -      True     0.100
                                    LP     -      False      NaN
                                    TH     -      False      NaN
                                    VISam  L2/3   False      NaN
                                    VISl   L1     False      NaN
                                                           ...  
799864342  static_gratings    63    LP     -      False      NaN
                                    VISp   L6     True     0.150
                              64    LP     -      False      NaN
                              65    LP     -      True       NaN
                              66    LP     -      False    0.015
Name: idxmax, Length: 171371, dtype: float64

In [20]:
f, ax = plt.subplots(figsize=(4, 4), tight_layout=True)
sns.boxplot(
    x='idxmax', y='dfnmax', color='gray', ax=ax,
    data=pd.concat([idxmax, dfnmax], axis=1), showfliers=False
)
ax.set_xlabel('optimal time (s)')
ax.set_ylabel('max differentiation');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
# f, ax = plt.subplots(figsize=(5, 3.5), tight_layout=True)
# sns.stripplot(
#     x='idxmax', y='dfnmax', hue='stimulus_name', size=1, dodge=True, ax=ax,
#     data=pd.concat([idxmax, dfnmax], axis=1).reset_index('stimulus_name'),
# )
# ax.set_xlabel('optimal time (s)')
# ax.set_ylabel('max differentiation')
# # ax.set_xscale('log')
# ax.set_yscale('log')
# ax.legend(loc=(1.005, 0), fontsize=6);

## Optimal time vs mean firing rate

In [22]:
# f, ax = plt.subplots(figsize=(5, 3.5), tight_layout=True)
# sns.stripplot(
#     x='idxmax', y='mean firing rate', hue='layer', size=1, dodge=True, ax=ax,
#     data=pd.concat([idxmax, mfrs], axis=1).reset_index('layer'),
# )
# ax.set_xlabel('optimal time (s)')
# # ax.set_xscale('log')
# ax.set_yscale('log')
# ax.legend(loc=(1.005, 0), fontsize=6);

## What fraction of units show an optimum?

In [23]:
n_opt = idxmax.groupby('stimulus_name').apply(
    lambda df: df.groupby(['area', 'FS_RS']).apply(
        lambda s: len(s.dropna())
    ).unstack().rename(columns={False:'FS', True:'RS'})#.mean()
)

n_tot = idxmax.groupby('stimulus_name').apply(
    lambda df: df.groupby(['area', 'FS_RS']).apply(
        lambda s: len(s)
    ).unstack().rename(columns={False:'FS', True:'RS'})#.mean()
)

frac_opt = n_opt / n_tot

frac_opt

FS_RS                        FS        RS
stimulus_name   area                     
dot_motion      LGd    0.450000  0.548913
                LP     0.377926  0.548951
                TH     0.375000  0.611765
                VISal  0.381481  0.294815
                VISam  0.332046  0.344376
...                         ...       ...
static_gratings VISam  0.536145  0.366252
                VISl   0.546296  0.380952
                VISp   0.650407  0.313028
                VISpm  0.529412  0.439894
                VISrl  0.488636  0.280365

[117 rows x 2 columns]

In [24]:
# average number of neurons per area per stimulus
n_tot.groupby('stimulus_name').mean().mean()

FS_RS
FS     273.555556
RS    1191.153846
dtype: float64

In [25]:
# average number of neurons with optimal timescale per area per stimulus
n_opt.groupby('stimulus_name').mean().mean()

FS_RS
FS    113.034188
RS    379.145299
dtype: float64

In [26]:
# cortical RS neurons only
display(n_tot.swaplevel().loc[region_sets['VisCtx']].groupby('stimulus_name').mean().mean())
display(n_opt.swaplevel().loc[region_sets['VisCtx']].groupby('stimulus_name').mean().mean())

FS_RS
FS     300.820513
RS    1429.589744
dtype: float64

FS_RS
FS    121.500000
RS    373.512821
dtype: float64

In [27]:
# fraction of neurons that have an optimal timescale per area per stimulus
frac_opt.groupby('stimulus_name').mean().mean()

FS_RS
FS    0.423386
RS    0.366012
dtype: float64

In [28]:
# fraction optimal in individual areas per stimulus (since all expts do not have all stimuli)
frac_opt.groupby('area').mean().reindex(relevant_regions)

FS_RS,FS,RS
area,,
LGd,0.501898,0.607807
LP,0.430317,0.519791
TH,0.414594,0.565390
VISp,0.432148,0.240281
VISl,0.382787,0.252359
VISrl,0.409792,0.220870
VISal,0.412785,0.275134
VISpm,0.411098,0.307791
VISam,0.415055,0.304686


In [29]:
layers = ['L1', 'L2/3', 'L4', 'L5', 'L6']
# what fraction of units has an optimal timescale?
f, (ax, ax2) = plt.subplots(
    1, 2, figsize=(5.5, 2.), tight_layout=True,
    sharey=True, gridspec_kw=dict(width_ratios=[1, 0.3])
)

f_optimal = idxmax.groupby('stimulus_name').apply(
    lambda _s: _s.groupby(['area', 'FS_RS']).apply(
        lambda s: len(s.dropna())/len(s)
    ).unstack().rename(columns={False:'FS', True:'RS'})
).swaplevel().sort_index()
f_optimal = f_optimal.loc[[
    c for c in hierarchy if c in f_optimal.index.levels[0]
]]

f_optimal.groupby('area').mean().plot(
    yerr=f_optimal.groupby('area').std(), marker='o', ax=ax
)
ax.set_ylabel('fraction optimal', fontsize=8)
ax.set_xticks(range(len(f_optimal.index.levels[0])))
ax.set_xticklabels(f_optimal.index.levels[0], rotation=30, )
ax.legend(fontsize=7, frameon=False)
ax.set_xlabel('area along hierarchy', fontsize=8)

f_optimal = idxmax.groupby('stimulus_name').apply(
    lambda _s: _s.groupby(['layer', 'FS_RS']).apply(
        lambda s: len(s.dropna())/len(s)
    ).unstack().rename(columns={False:'FS', True:'RS'})
).swaplevel().sort_index()
f_optimal = f_optimal.loc[[
    c for c in layers if c in f_optimal.index.levels[0]
]]

f_optimal.groupby('layer').mean().plot(
    yerr=f_optimal.groupby('layer').std(), marker='o',
    ax=ax2, legend=False
)
ax2.set_ylabel('fraction optimal', fontsize=8)
ax2.set_xticks(range(len(f_optimal.index.remove_unused_levels().levels[0])))
ax2.set_xticklabels(f_optimal.index.remove_unused_levels().levels[0])
ax2.set_xlabel('layer', fontsize=8)

for ax0 in [ax, ax2]:
    ax0.tick_params(axis='both', labelsize=7)

f.align_xlabels()
f.savefig('fig_supp_timescales.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
stim_diff = pd.read_pickle(path.join(data_directory, 'spectral_differentiation_stimulus.pkl'))
stim_diff = stim_diff / stim_diff.columns.get_level_values(1)**2
msd = stim_diff.reorder_levels([2, 0, 1], axis=1).mean_diff.groupby(level=1, axis=1).mean()
msd

,0.005,0.010,0.015,0.020,0.030,0.060,0.100,0.150,0.300,0.500,1.000,3.000,10.000
drifting_gratings,75.522055,75.084092,74.546305,73.861366,72.393346,64.288436,56.152077,49.127052,32.776354,16.314573,10.001740,6.477853,2.682836
drifting_gratings_contrast,0.000000,0.000000,0.000000,0.000000,0.000000,0.763849,1.001863,1.967366,3.356052,3.021572,4.190317,3.383678,1.743942
flashes,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.770216,7.846824,24.737785,12.679904,3.677947
gabors,11.501625,11.501636,11.455716,11.455744,11.374635,11.091270,11.068019,10.240929,8.740839,7.967084,5.559245,3.271730,1.777082
natural_movie_one,64.866778,64.858514,64.834847,64.799274,64.708111,64.261180,63.659275,62.571709,60.098324,57.576342,52.778192,51.622842,42.397623
natural_movie_one_shuffled,59.728118,59.341519,58.673586,57.677089,54.948894,44.802448,36.540143,30.080919,21.740530,16.566495,11.642560,6.806867,4.472170
natural_movie_three,59.328546,59.304894,59.258390,59.193031,59.032847,58.274465,57.328008,55.602310,51.834834,48.226178,42.525704,37.789186,26.006905
static_gratings,102.739963,102.740001,102.713635,102.713587,102.652584,102.252796,102.266474,91.408513,80.433328,73.280649,51.221907,29.797235,16.488908


In [31]:
f, ax = plt.subplots(figsize=(4, 5), tight_layout=True)
msd.T.plot(ax=ax)
ax.set_xscale('log')
ax.set_yscale('log', nonpositive='mask');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
def find_knee(r):
    r = np.log(r).replace([np.inf, -np.inf], np.nan)
    r.index = np.log(r.index)
    x, y = r.dropna().index[-3:], r.dropna().values[-3:]
#     print(r.name, x, y)
    m1, c1 = np.polyfit(x, y, 1)
    x, y = r.dropna().index[:2], r.dropna().values[:2]
#     print(r.name, x, y)
    m2, c2 = np.polyfit(x, y, 1)
    return np.exp((c2-c1)/(m1-m2))

stim_opt_t = msd.apply(find_knee, axis=1)
stim_opt_t

/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


drifting_gratings             0.034173
drifting_gratings_contrast    1.357472
flashes                       1.361311
gabors                        0.232832
natural_movie_one             0.158985
natural_movie_one_shuffled    0.018633
natural_movie_three           0.259575
static_gratings               0.243010
dtype: float64

In [33]:
# stim_opt_t = {
#     'drifting_gratings':0.05,
#     'drifting_gratings_contrast':1,
#     'flashes':1,
#     'gabors':0.25,
#     'natural_movie_one':0.15,
#     'natural_movie_one_shuffled':0.025,
#     'natural_movie_three':0.09,
#     'static_gratings':0.25
# }

In [50]:
def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy
    See https://zhiyzuo.github.io/Pearson-Correlation-CI-in-Python for reference
    Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''

    r, p = sp.stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(len(x)-3)
    z = sp.stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return dict(r=r, p=p, ci_low=lo, ci_high=hi)

def mkplt_mot(ax, RS_CELLS=True, grouper='area'):
    if grouper=='area':
        groups = ['stim', 'LGd', 'LP', 'VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam']
    else:
        groups = ['L1', 'L2/3', 'L4', 'L5', 'L6']
    groupwise_optima = idxmax.xs(
        RS_CELLS, level='FS_RS'
    ).groupby(grouper).apply(
        lambda df: df.groupby('stimulus_name').mean().sort_values()
    ).unstack(grouper)
    groupwise_optima_sd = idxmax.xs(
        RS_CELLS, level='FS_RS'
    ).groupby(grouper).apply(
        lambda df: df.groupby('stimulus_name').sem().sort_values()
    ).unstack(grouper)
    groupwise_optima_sd['stim'] = 0
    groupwise_optima = groupwise_optima.join(
        pd.Series(stim_opt_t, name='stim')
    )[groups]#[[c for c in ecephys.hierarchy.keys() if c in areawise_optima.columns]]

    def _unfold(lrr):
        return pd.Series(lrr)
#         return pd.Series(
#             lrr, index=['slope', 'intercept', 'rvalue', 'pvalue', 'stderr']
#         )
    
    if grouper=='area':
        print(f'stats table for cortical hierarchy dependence ({"RS" if RS_CELLS else "FS"}):')
        opt_ts_inc = groupwise_optima[region_sets['VisCtx']].apply(
            lambda r: _unfold(
                pearsonr_ci(range(len(region_sets['VisCtx'])), r.values)
#                 sp.stats.linregress(range(len(region_sets['VisCtx'])), r.values)
            ), axis=1
        )
        pd.set_option("display.precision", 2)
        display(opt_ts_inc)
        pd.set_option("display.precision", 8)
    
    for stim in [
        'flashes', 'gabors', 'drifting_gratings', 'static_gratings',
        'natural_movie_one', 'natural_movie_three', 'natural_movie_one_shuffled'
    ]:
        if grouper=='area':
            label = f'({opt_ts_inc["p"][stim]:.2f}) {stim}'
        else:
            label=''
        ax.fill_between(
            groups,
            groupwise_optima.loc[stim, groups]-groupwise_optima_sd.loc[stim, groups],
            groupwise_optima.loc[stim, groups]+groupwise_optima_sd.loc[stim, groups],
            alpha=0.4, label=label
        )
    ax.set_title(f'{"RS" if RS_CELLS else "FS"} cells only', fontsize=8)
    ax.tick_params(labelsize=7)
#     sns.despine(ax=ax)
    ax.label_outer()

In [51]:
layers = ['L1', 'L2/3', 'L4', 'L5', 'L6']
areas = ['stim', 'LGd', 'LP', 'VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam']

# with sns.axes_style('white'):
f = plt.figure(figsize=(5.5, 4), tight_layout=True)
gs = plt.GridSpec(2, 2, figure=f, width_ratios=[8, 5])
ax0 = f.add_subplot(gs[0, 0])
ax1 = f.add_subplot(gs[0, 1], sharey=ax0)
ax2 = f.add_subplot(gs[1, 0], sharey=ax0, sharex=ax0)
ax3 = f.add_subplot(gs[1, 1], sharey=ax0, sharex=ax1)

mkplt_mot(ax0)
mkplt_mot(ax2, RS_CELLS=False)
mkplt_mot(ax1, grouper='layer')
mkplt_mot(ax3, RS_CELLS=False, grouper='layer')

f.text(0.005, 0.5, 'mean optimal timescale (s)', va='center', ha='left', fontsize=8, rotation=90)
ax0.legend(fontsize=5, ncol=2, loc=3, frameon=False)
ax2.set_xticks(range(len(areas)))
ax2.set_xticklabels(areas)
ax3.set_xticks(range(len(layers)))
ax3.set_xticklabels(layers)
ax0.set_ylim(0.04, 0.34);
ax0.set_yticks([0.0, 0.1, 0.15, 0.2, 0.25, 0.3])
ax2.set_yticks([0.05, 0.1, 0.15, 0.2, 0.25, 0.3])

f.savefig('fig_supp_timescales_mot.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

stats table for cortical hierarchy dependence (RS):


,r,p,ci_low,ci_high
stimulus_name,,,,
dot_motion,-0.05,0.92,-0.83,0.79
drifting_gratings,-0.71,0.12,-0.96,0.25
drifting_gratings_75_repeats,-0.24,0.65,-0.88,0.71
drifting_gratings_contrast,0.26,0.62,-0.70,0.89
flashes,-0.72,0.11,-0.97,0.22
gabors,0.33,0.53,-0.66,0.90
natural_movie_one,-0.05,0.93,-0.83,0.80
natural_movie_one_more_repeats,0.76,0.08,-0.13,0.97
natural_movie_one_shuffled,-0.70,0.12,-0.96,0.25


stats table for cortical hierarchy dependence (FS):


,r,p,ci_low,ci_high
stimulus_name,,,,
dot_motion,-0.84,0.04,-0.98,-0.07
drifting_gratings,0.07,0.90,-0.79,0.83
drifting_gratings_75_repeats,0.62,0.19,-0.38,0.95
drifting_gratings_contrast,-0.80,0.05,-0.98,0.02
flashes,-0.81,0.05,-0.98,0.02
gabors,-0.82,0.04,-0.98,-0.04
natural_movie_one,0.54,0.27,-0.48,0.94
natural_movie_one_more_repeats,-0.10,0.85,-0.84,0.77
natural_movie_one_shuffled,-0.74,0.09,-0.97,0.17


In [40]:
layers = ['L1', 'L2/3', 'L4', 'L5', 'L6']
areas = ['stim', 'LGd', 'LP', 'VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam']

# with sns.axes_style('white'):
f = plt.figure(figsize=(5.5, 4), tight_layout=True)
gs = plt.GridSpec(2, 2, figure=f, width_ratios=[8, 5])
ax0 = f.add_subplot(gs[0, 0])
ax1 = f.add_subplot(gs[0, 1], sharey=ax0)
ax2 = f.add_subplot(gs[1, 0], sharey=ax0, sharex=ax0)
ax3 = f.add_subplot(gs[1, 1], sharey=ax0, sharex=ax1)

mkplt_mot(ax0)
mkplt_mot(ax2, RS_CELLS=False)
mkplt_mot(ax1, grouper='layer')
mkplt_mot(ax3, RS_CELLS=False, grouper='layer')

f.text(0.005, 0.5, 'mean optimal timescale (s)', va='center', ha='left', fontsize=8, rotation=90)
ax0.legend(fontsize=5, ncol=2, loc=3, frameon=False)
ax2.set_xticks(range(len(areas)))
ax2.set_xticklabels(areas)
ax3.set_xticks(range(len(layers)))
ax3.set_xticklabels(layers)
ax0.set_ylim(0.04, 0.34);
ax0.set_yticks([0.0, 0.1, 0.15, 0.2, 0.25, 0.3])
ax2.set_yticks([0.05, 0.1, 0.15, 0.2, 0.25, 0.3])

f.savefig('fig_supp_timescales_mot.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

stats table for cortical hierarchy dependence (RS):


,slope,intercept,rvalue,pvalue,stderr
stimulus_name,,,,,
dot_motion,-0.000271,0.112023,-0.050796,0.923871,0.002663
drifting_gratings,-0.003040,0.117653,-0.706246,0.116763,0.001524
drifting_gratings_75_repeats,-0.001999,0.116466,-0.237787,0.650041,0.004082
drifting_gratings_contrast,0.001480,0.152644,0.261596,0.616557,0.002730
flashes,-0.003860,0.207221,-0.721318,0.105674,0.001853
gabors,0.001649,0.145730,0.325603,0.528856,0.002394
natural_movie_one,-0.000301,0.108815,-0.045940,0.931139,0.003271
natural_movie_one_more_repeats,0.004108,0.081952,0.759888,0.079559,0.001757
natural_movie_one_shuffled,-0.007910,0.176614,-0.703066,0.119165,0.004000


stats table for cortical hierarchy dependence (FS):


,slope,intercept,rvalue,pvalue,stderr
stimulus_name,,,,,
dot_motion,-0.004557,0.114386,-0.835259,0.038474,0.001500
drifting_gratings,0.000328,0.092565,0.066071,0.901038,0.002479
drifting_gratings_75_repeats,0.004022,0.081460,0.620553,0.188654,0.002541
drifting_gratings_contrast,-0.010354,0.189137,-0.803740,0.053997,0.003832
flashes,-0.009415,0.226675,-0.806158,0.052720,0.003455
gabors,-0.006714,0.168755,-0.824593,0.043453,0.002303
natural_movie_one,0.004425,0.093214,0.542834,0.265727,0.003423
natural_movie_one_more_repeats,-0.000511,0.087981,-0.099185,0.851711,0.002561
natural_movie_one_shuffled,-0.007795,0.128550,-0.742732,0.090766,0.003513


In [36]:
types = ['FS', 'RS']
typewise_optima = idxmax.groupby('FS_RS').apply(lambda df: df.groupby('stimulus_name').mean().sort_values()).unstack('FS_RS')
typewise_optima_sd = idxmax.groupby('FS_RS').apply(lambda df: df.groupby('stimulus_name').sem().sort_values()).unstack('FS_RS')

f, ax = plt.subplots(figsize=(2, 3), tight_layout=True)
for stim in ['flashes', 'gabors', 'drifting_gratings', 'static_gratings', 'natural_movie_one', 'natural_movie_three', 'natural_movie_one_shuffled']:#areawise_optima.index:
    ax.fill_between(
        ['FS', 'RS'],
        typewise_optima.loc[stim, :]-typewise_optima_sd.loc[stim, :],
        typewise_optima.loc[stim, :]+typewise_optima_sd.loc[stim, :],
        alpha=0.4, label=stim
    )
# typewise_optima.T.plot(ax=ax)
ax.set_ylabel('mean optimal timescale')
# ax.legend(loc=(1.01, 0), fontsize=7)
ax.set_xticks(range(len(types)))
ax.set_xticklabels(['FS', 'RS']);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---